In [47]:
import pandas as pd
import numpy as np



In [48]:
df_template = pd.DataFrame(columns=["driver_id, driver_ref, q1_position, q2_position, q3_position, circuit, constructor_standing, driver_standing, year, constructor, quali_position"])

In [99]:
drivers = pd.read_csv("archive/drivers.csv")
qualifying = pd.read_csv("archive/qualifying.csv")
constructor_results = pd.read_csv("archive/constructor_results.csv")
race_results = pd.read_csv("archive/results.csv")
constructors = pd.read_csv("archive/constructors.csv")
races = pd.read_csv("archive/races.csv")

In [50]:
driver_ids_list = [1, 4, 853, 854, 822, 830, 815, 847, 849, 846, 817, 839, 840, 20, 852, 842, 841, 8, 832, 844]
drivers_clean = drivers[drivers.driverId.isin(driver_ids_list)].drop(["number", "forename", "surname", "dob", "nationality", "url", "driverRef"], axis=1).reset_index(drop=True)

drivers_clean

,driverId,code
0,1,HAM
1,4,ALO
2,8,RAI
3,20,VET
4,842,GAS
5,815,PER
6,817,RIC
7,822,BOT
8,830,VER
9,832,SAI


In [51]:
qualifying

,qualifyId,raceId,driverId,constructorId,number,position,q1,q2,q3
0,1,18,1,1,22,1,1:26.572,1:25.187,1:26.714
1,2,18,9,2,4,2,1:26.103,1:25.315,1:26.869
2,3,18,5,1,23,3,1:25.664,1:25.452,1:27.079
3,4,18,13,6,2,4,1:25.994,1:25.691,1:27.178
4,5,18,2,2,3,5,1:25.960,1:25.518,1:27.236
...,...,...,...,...,...,...,...,...,...
8769,8810,1055,852,213,22,16,1:18.556,NaN,NaN
8770,8811,1055,8,51,7,17,1:18.917,NaN,NaN
8771,8812,1055,854,210,47,18,1:19.117,NaN,NaN
8772,8813,1055,849,3,6,19,1:19.219,NaN,NaN


In [52]:
qualifying = qualifying.drop(["number", "qualifyId", "position"], axis=1)

In [53]:
test1 = qualifying[["raceId", "driverId", "constructorId"]].copy()


In [54]:
test2 = qualifying.drop(["q2", "q3"], axis=1)
test2 = test2.dropna()
test2["q1_position"] = test2.sort_values(["q1"]).groupby("raceId").cumcount() + 1

In [55]:
test3 = qualifying.drop(["q1", "q3"], axis=1)
test3 = test3.dropna()
test3["q2_position"] = test3.sort_values(["q2"]).groupby("raceId").cumcount() + 1

In [56]:
test4 = qualifying.drop(["q1", "q2"], axis=1)
test4 = test4.dropna()
test4["q3_position"] = test4.sort_values(["q3"]).groupby("raceId").cumcount() + 1

In [57]:
test1 = test1.merge(
    test2,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [58]:
test1 = test1.merge(
    test3,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

test1 = test1.merge(
    test4,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [59]:
cleaned_positions = test1.drop(["q1", "q2", "q3"], axis=1)

In [60]:
test_x = cleaned_positions.groupby(["raceId"]).transform(lambda x: x.fillna(x.max() + 1))
test_x["raceId"] = cleaned_positions["raceId"]

In [69]:
qualifying = test_x

In [70]:
qualifying

,driverId,constructorId,q1_position,q2_position,q3_position,raceId
0,1,1,10.0,1.0,1.0,18
1,9,2,4.0,2.0,2.0,18
2,5,1,1.0,3.0,3.0,18
3,13,6,3.0,5.0,4.0,18
4,2,2,2.0,4.0,5.0,18
...,...,...,...,...,...,...
8769,852,213,16.0,16.0,11.0,1055
8770,8,51,17.0,16.0,11.0,1055
8771,854,210,18.0,16.0,11.0,1055
8772,849,3,19.0,16.0,11.0,1055


In [71]:
quali_w_results = qualifying.merge(
    race_results,
    on=["raceId", "driverId", "constructorId"],
    how="left"
)

In [72]:
# quali_results = quali_results.drop(["resultId", ""])
quali_w_results[quali_w_results.raceId == 1040]

,driverId,constructorId,q1_position,q2_position,q3_position,raceId,resultId,number,grid,position,...,positionOrder,points,laps,time,milliseconds,fastestLap,rank,fastestLapTime,fastestLapSpeed,statusId
8534,1,131,2.0,4.0,1.0,1040,24808.0,44.0,1.0,3.0,...,3.0,15.0,53.0,+22.729,5663093.0,48.0,7.0,1:38.075,214.660,1.0
8535,830,9,5.0,9.0,2.0,1040,24807.0,33.0,2.0,2.0,...,2.0,18.0,53.0,+7.729,5648093.0,53.0,3.0,1:37.332,216.298,1.0
8536,822,131,1.0,2.0,3.0,1040,24806.0,77.0,3.0,1.0,...,1.0,26.0,53.0,1:34:00.364,5640364.0,51.0,1.0,1:37.030,216.972,1.0
8537,815,211,7.0,5.0,4.0,1040,24809.0,11.0,4.0,4.0,...,4.0,12.0,53.0,+30.558,5670922.0,52.0,8.0,1:38.141,214.515,1.0
8538,817,4,6.0,1.0,5.0,1040,24810.0,3.0,5.0,5.0,...,5.0,10.0,53.0,+52.065,5692429.0,53.0,6.0,1:37.886,215.074,1.0
8539,832,1,12.0,3.0,6.0,1040,24824.0,55.0,6.0,NaN,...,19.0,0.0,0.0,NaN,NaN,NaN,0.0,NaN,NaN,3.0
8540,839,4,4.0,10.0,7.0,1040,24812.0,31.0,7.0,7.0,...,7.0,6.0,53.0,+1:08.006,5708370.0,48.0,13.0,1:39.216,212.191,1.0
8541,846,1,9.0,6.0,8.0,1040,24820.0,4.0,8.0,15.0,...,15.0,0.0,52.0,NaN,NaN,50.0,5.0,1:37.377,216.198,11.0
8542,842,213,8.0,7.0,9.0,1040,24814.0,10.0,9.0,9.0,...,9.0,2.0,53.0,+1:29.766,5730130.0,50.0,2.0,1:37.231,216.523,1.0
8543,848,9,11.0,8.0,10.0,1040,24815.0,23.0,15.0,10.0,...,10.0,1.0,53.0,+1:37.860,5738224.0,51.0,9.0,1:38.377,214.001,1.0


In [73]:
quali_w_results = quali_w_results.drop(["resultId", "number", "grid", "positionText",
                      "positionOrder", "points","laps", "time",
                      "milliseconds", "fastestLap", "rank",
                      "fastestLapTime", "fastestLapSpeed", "statusId"], axis=1)

quali_w_results

,driverId,constructorId,q1_position,q2_position,q3_position,raceId,position
0,1,1,10.0,1.0,1.0,18,1.0
1,9,2,4.0,2.0,2.0,18,NaN
2,5,1,1.0,3.0,3.0,18,5.0
3,13,6,3.0,5.0,4.0,18,NaN
4,2,2,2.0,4.0,5.0,18,2.0
...,...,...,...,...,...,...,...
8769,852,213,16.0,16.0,11.0,1055,NaN
8770,8,51,17.0,16.0,11.0,1055,12.0
8771,854,210,18.0,16.0,11.0,1055,18.0
8772,849,3,19.0,16.0,11.0,1055,16.0


In [89]:
quali_w_results_clean = quali_w_results.copy()

In [90]:
quali_w_results_clean = quali_w_results_clean.groupby(["raceId"]).transform(lambda x: x.fillna(x.max() + 1))
quali_w_results_clean["raceId"] = quali_w_results["raceId"]

In [91]:
quali_w_results_clean

,driverId,constructorId,q1_position,q2_position,q3_position,position,raceId
0,1,1,10.0,1.0,1.0,1.0,18
1,9,2,4.0,2.0,2.0,9.0,18
2,5,1,1.0,3.0,3.0,5.0,18
3,13,6,3.0,5.0,4.0,9.0,18
4,2,2,2.0,4.0,5.0,2.0,18
...,...,...,...,...,...,...,...
8769,852,213,16.0,16.0,11.0,20.0,1055
8770,8,51,17.0,16.0,11.0,12.0,1055
8771,854,210,18.0,16.0,11.0,18.0,1055
8772,849,3,19.0,16.0,11.0,16.0,1055


In [96]:
quali_w_results_clean = quali_w_results_clean.rename(columns=
    {
        "position": "driver_standing"
    }
)

In [106]:
#merging in constructor results
results = quali_w_results_clean.copy()
results_w_races = results.merge(
    races,
    on="raceId",
    how="left"
)

In [107]:
results_w_races = results_w_races.drop(["round", "name", "date", "time", "url"], axis=1)
results_w_races


,driverId,constructorId,q1_position,q2_position,q3_position,driver_standing,raceId,year,circuitId
0,1,1,10.0,1.0,1.0,1.0,18,2008,1
1,9,2,4.0,2.0,2.0,9.0,18,2008,1
2,5,1,1.0,3.0,3.0,5.0,18,2008,1
3,13,6,3.0,5.0,4.0,9.0,18,2008,1
4,2,2,2.0,4.0,5.0,2.0,18,2008,1
...,...,...,...,...,...,...,...,...,...
8769,852,213,16.0,16.0,11.0,20.0,1055,2021,4
8770,8,51,17.0,16.0,11.0,12.0,1055,2021,4
8771,854,210,18.0,16.0,11.0,18.0,1055,2021,4
8772,849,3,19.0,16.0,11.0,16.0,1055,2021,4


In [132]:
constructors_clean = constructor_results.copy()
constructors_clean = constructors_clean.drop(["constructorResultsId", "status"], axis=1)
# constructors_clean = constructors_clean.rename(
#     columns={
#         "points": "constructor_standing"
#     }
# )

In [133]:
constructors_clean["constructor_standing"] = constructors_clean.sort_values(["points"]).groupby("raceId").cumcount() + 1
constructors_clean = constructors_clean.drop(["points"], axis=1)

In [134]:
final_data = results_w_races.merge(
    constructors_clean,
    on=["raceId", "constructorId"],
    how="left"
)

In [135]:
final_data = final_data.merge(
    drivers_clean,
    on="driverId",
    how="right"
)

final_data.columns = ["driver_id", "constructor_id",
                      "q1_position", "q2_position",
                      "q3_position", "driver_standing", "race_id",
                      "year", "circuit_id", "constructor_standing",
                      "driver_code"]

In [136]:
final_data


,driver_id,constructor_id,q1_position,q2_position,q3_position,driver_standing,race_id,year,circuit_id,constructor_standing,driver_code
0,1,1,10.0,1.0,1.0,1.0,18,2008,1,11.0,HAM
1,1,1,4.0,3.0,4.0,5.0,19,2008,2,10.0,HAM
2,1,1,4.0,5.0,3.0,13.0,20,2008,3,9.0,HAM
3,1,1,4.0,7.0,5.0,3.0,21,2008,4,10.0,HAM
4,1,1,2.0,6.0,3.0,2.0,22,2008,5,8.0,HAM
...,...,...,...,...,...,...,...,...,...,...,...
2367,853,210,20.0,16.0,11.0,19.0,1055,2021,4,4.0,MAZ
2368,854,210,19.0,16.0,11.0,16.0,1052,2021,3,1.0,MSC
2369,854,210,18.0,16.0,10.0,16.0,1053,2021,21,2.0,MSC
2370,854,210,19.0,16.0,11.0,17.0,1054,2021,75,1.0,MSC
